In [1]:
import pandas as pd
path_goods = '/home/guozijun/@Data Mining/VIPcom/goods_train.txt'
path_users = '/home/guozijun/@Data Mining/VIPcom/user_action_train.txt'

df_users = pd.read_table(path_users,header=None)
df_goods = pd.read_table(path_goods,header = None)

df_users.columns = ['uid','spu_id','action_type','date']
df_goods.columns = ['spu_id','brand_id','cat_id']

df = pd.merge(df_users, df_goods, how='left', on ='spu_id')

T = df['date']
time_list = list(set(T))
time_dict = dict(zip(time_list,range(0,len(time_list))))
df['date'] = [time_dict[x] for x in df['date']]

path_re =  '/home/guozijun/@Data Mining/VIPcom/users_goods_train.csv'
df.to_csv(path_re,index=False)
print 'done'
print df.isnull().sum()
df.head()

done
uid            0
spu_id         0
action_type    0
date           0
brand_id       0
cat_id         0
dtype: int64


,uid,spu_id,action_type,date,brand_id,cat_id
0,522945,338312,0,7,10005188,1012
1,320510,839213,0,43,10010631,271
2,314210,369282,0,81,10001351,311
3,381441,730484,0,81,10000866,311
4,168858,902515,0,43,10028088,297


In [2]:
import pandas as pd
path =  '/home/guozijun/@Data Mining/VIPcom/users_goods_train.csv'
df= pd.read_csv(path)
print 'download data'

action =pd.get_dummies(df['action_type'],prefix = 'action_type')
df = pd.concat([df,action],axis=1)
del df['action_type']
df.head()

download data


,uid,spu_id,date,brand_id,cat_id,action_type_0,action_type_1
0,522945,338312,7,10005188,1012,1,0
1,320510,839213,43,10010631,271,1,0
2,314210,369282,81,10001351,311,1,0
3,381441,730484,81,10000866,311,1,0
4,168858,902515,43,10028088,297,1,0


In [3]:
import numpy as np
def user_purchase_spu(start_date,end_date,df):
    df_train = df.loc[(df['date']>=start_date) & (df['date']<end_date)]
    df_train = df_train[['uid','spu_id','action_type_0','action_type_1']]
    df_train = df_train.groupby(['uid','spu_id'],as_index = False).sum()
    
    list_b = df_train['action_type_0'].values   
    list_b = [float(i) for i in list_b]
    list_p = df_train['action_type_1'].values   #用户对商品 点击购买的次数
    list_p = [float(i) for i in list_p]
                           
    
    list_p =np.array(list_p)
    list_b = np.array(list_b)
    
    list_b = list_b+list_p  #用户对商品 点击的次数
    
    list_p_r = list_p/list_b
    
    df_train['u_buy_s_n'] = list_p
    df_train['u_buy_s_r'] = list_p_r
    df_train['u_browse_s_n'] = list_b
    
    df_u_b = df_train[['uid','action_type_1','action_type_0']]
    df_u_b = df_u_b.groupby('uid',as_index = False).sum()
    
    df_u_b['buy_number'] = df_u_b['action_type_1'].values
    df_u_b['browse_number'] =( df_u_b['action_type_0'].values+df_u_b['action_type_1'].values)
    
    del df_u_b['action_type_0']
    del df_u_b['action_type_1']

    df_train =pd.merge(df_train, df_u_b, how = 'left',on='uid')
    list_browse_all = df_train['browse_number']
    list_browse_all = [float(i) for i in list_browse_all]
    
    list_buy_all = df_train['buy_number']
    
    df_train['u_s_perference1'] = list_p/list_buy_all
    df_train['u_s_perference2'] = (list_p+list_b)/list_browse_all
    
    df_train['u_s_perference1'].fillna(0,inplace=True)

        

    del df_train['buy_number']
    del df_train['browse_number']
    del df_train['action_type_0']
    del df_train['action_type_1']

    return df_train


def user_purchase_cat(start_date,end_date,df):
    df_train = df.loc[(df['date']>=start_date) & (df['date']<end_date)]
    df_train = df_train[['uid','cat_id','action_type_0','action_type_1']]
    df_train = df_train.groupby(['uid','cat_id'],as_index = False).sum()
    
    list_b = df_train['action_type_0'].values   
    list_b = [float(i) for i in list_b]
    list_p = df_train['action_type_1'].values   #用户对商品 点击购买的次数
    list_p = [float(i) for i in list_p]
                           
    
    list_p =np.array(list_p)
    list_b = np.array(list_b)
    
    list_b = list_b+list_p  #用户对商品 点击的次数
    
    list_p_r = list_p/list_b
    
    df_train['u_buy_c_n'] = list_p
    df_train['u_buy_c_r'] = list_p_r
    df_train['u_browse_c_n'] = list_b
    
    df_u_b = df_train[['uid','action_type_1','action_type_0']]
    df_u_b = df_u_b.groupby('uid',as_index = False).sum()
    
    df_u_b['buy_number'] = df_u_b['action_type_1'].values
    df_u_b['browse_number'] =( df_u_b['action_type_0'].values+df_u_b['action_type_1'].values)
    
    del df_u_b['action_type_0']
    del df_u_b['action_type_1']

    df_train =pd.merge(df_train, df_u_b, how = 'left',on='uid')
    list_browse_all = df_train['browse_number']
    list_browse_all = [float(i) for i in list_browse_all]
    
    list_buy_all = df_train['buy_number']
    
    df_train['u_c_perference1'] = list_p/list_buy_all
    df_train['u_c_perference2'] = (list_p+list_b)/list_browse_all
    
    df_train['u_c_perference1'].fillna(0,inplace=True)

        

    del df_train['buy_number']
    del df_train['browse_number']
    del df_train['action_type_0']
    del df_train['action_type_1']

    return df_train


def get_feature(start_date,end_date,df):
    df_X = df.loc[(df['date']>=start_date) & (df['date']<end_date)]
    df_X = df_X[['uid','spu_id','action_type_1']]
    df_X = df_X.groupby(['uid','spu_id'],as_index =False).sum()
    df_X = df_X[['uid','spu_id']]

    df_y = df.loc[(df['date']>=end_date) & (df['date']<end_date+7)]
    df_y = df_y[['uid','spu_id','action_type_1']]
    
    df_y = df_y.groupby(['uid','spu_id'],as_index =False).sum()

    df_X_y = pd.merge(df_X,df_y,how = 'left',on = ['uid','spu_id'])

    df_X_y['action_type_1'].fillna(0,inplace = True)

    return df_X_y



In [5]:
df_X_y = 
(0,7,df)
print df_X_y.isnull().sum()
df_X_y.head()

uid              0
spu_id           0
action_type_1    0
dtype: int64


,uid,spu_id,action_type_1
0,3,79517,0.0
1,3,114037,0.0
2,3,189129,0.0
3,3,226275,0.0
4,3,246204,0.0


In [7]:
df_X_y['action_type_1'].value_counts()

0.0    5106273
1.0        788
Name: action_type_1, dtype: int64

In [9]:
df_train = user_purchase_cat(0,7,df)
print df_train.isnull().sum()
df_train.head()

uid                0
cat_id             0
u_buy_c_n          0
u_buy_c_r          0
u_browse_c_n       0
u_c_perference1    0
u_c_perference2    0
dtype: int64


,uid,cat_id,u_buy_c_n,u_buy_c_r,u_browse_c_n,u_c_perference1,u_c_perference2
0,3,271,0.0,0.0,2.0,0.0,0.111111
1,3,311,0.0,0.0,6.0,0.0,0.333333
2,3,1012,0.0,0.0,3.0,0.0,0.166667
3,3,1056,0.0,0.0,6.0,0.0,0.333333
4,3,7469,0.0,0.0,1.0,0.0,0.055556


In [10]:
df_train = user_purchase_spu(0,7,df)
print df_train.isnull().sum()
df_train.head()

uid                0
spu_id             0
u_buy_s_n          0
u_buy_s_r          0
u_browse_s_n       0
u_s_perference1    0
u_s_perference2    0
dtype: int64


,uid,spu_id,u_buy_s_n,u_buy_s_r,u_browse_s_n,u_s_perference1,u_s_perference2
0,3,79517,0.0,0.0,1.0,0.0,0.055556
1,3,114037,0.0,0.0,1.0,0.0,0.055556
2,3,189129,0.0,0.0,1.0,0.0,0.055556
3,3,226275,0.0,0.0,1.0,0.0,0.055556
4,3,246204,0.0,0.0,1.0,0.0,0.055556
